In [1]:
'''
本例实现的是书中图3-6所示的简单网络
X:输入数据集，每个零件的长度与质量
Y:输出数据集，零件是否合格
x1:零件长度
x2:零件质量
w1:input-lyout和hidden-lyout间的权重
w2:hidden-lyout和output-lyout间的权重
'''
import tensorflow as tf

# Numpy科学计算工具包，此处用于生成模拟数据集
from numpy.random import RandomState

# 定义训练数据batch的大小
batch_size = 8

# 定义神经网络的参数
w1 = tf.Variable(tf.random_normal([2,3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3,1], stddev=1, seed=1))

# 在shape的一个维度上使用None可以方便使用不同的batch大小
# 在训练时需要把数据分成比较小的batch，但是测试时，可以一次性使用全部的数据
# 当数据集比较小时，这样比较方便测试
# 但数据集比较大时，将大量数据放入一个batch可能会导致内存溢出
x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

# 定义神经网络前向传播的过程
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)
'''
# 使用非线性激活函数的前向传播
a = tf.nn.relu(tf.matmul(x, w1)+biases1)
y = tf.nn.relu(tf.matmul(a, w2)+biases2)
'''

# 定义损失函数和反向传播的算法
# cross_entropy交叉熵损失函数：H(p,q)=-Sigma[p*log(q)]
# tf.reduce_mean函数执行累加操作
# tf.log函数执行对数操作
# tf.clip_by_value函数可以将一个张量中的数值限制在一个范围内，避免一些运算错误(log0)
cross_entropy = -tf.reduce_mean(y_*tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
'''
# softmax回归之后的交叉熵损失函数
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(y, y_)
# 均方误差损失函数
mse = tf.reduce_mean(tf.square(y_-y))
# 带L2正则化的均方误差损失函数
# lambda为权重，tf自带函数为R(w)
loss = tf.reduce_mean(tf.square(y_-y)) + tf.contrib.layers.l2_reqularizer(lambda)(w)
'''
# train_step反向传播的优化方法
# 通过sess.run(train_step)就可以优化所有GraphKeys.TRAINABLE_VATIABLES集合[可学习的变量]中的变量，从而降低当前batch的损失函数
# 0.001学习率，.minimize()函数指定使用的损失函数
# tf.train.GradientDescentOptimizer
# tf.train.AdmOptimizer
# tf.train.MomentumOptimizer
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
'''
# 学习率的优化--指数衰减法
# tf.train.exponential_decay函数
global_step = tf.Variable(0)
# 通过exponential_decay函数生成学习率
# 初始学习率0.1 衰减系数0.96 衰减速度100
# staircase=True所以每训练100轮后学习率乘以0.96
learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.96, staircase=True)
# 使用指数衰减的学习率
# 在minimize函数中传入global_step将自动更新该参数，从而更新学习率
learning_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy, global_step=global_step)
'''

# 通过随机数生成一个模拟数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)

# 定义规则来给出样本的标签
# 定义所有x1+x2<1的样例都被认为时正样例，反之为负样例
# 使用0表示负样例，1表示正样例
Y = [[int(x1+x2<1)] for (x1,x2) in X]

# 创建会话运行Tensorflow程序
with tf.Session() as sess:# 通过with创建的session可以自行关闭，建议使用此方式
    init_op = tf.global_variables_initializer()
    #old method init_op = tf.initialize_all_variables()# 通过函数initialize_all_variables初始化所有变量，建议使用此方式
    # 执行初始化变量操作
    sess.run(init_op)
    print("w1=", sess.run(w1))
    print("w2=", sess.run(w2))
    
    # 设定训练的轮数
    STEPS = 5000
    for i in range(STEPS):
        # 每次选取batch_size个样本进行训练
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size, dataset_size)
        
        # 通过选取的样本训练神经网络并更新参数
        # feed_dict是一个字典，在字典中给出每个placeholder的取值
        sess.run(train_step, feed_dict={x:X[start:end], y_:Y[start:end]})
        if i%1000 == 0:
            # 每隔一段时间计算在所有数据上的交叉熵
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x:X, y_:Y})
            print("After %d training step(s), cross entropy on all data is %g"%(i, total_cross_entropy))
    
    # 打印调整后的权重
    print("w1=", sess.run(w1))
    print("w2=", sess.run(w2))

w1= [[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]
w2= [[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]
After 0 training step(s), cross entropy on all data is 0.0674925
After 1000 training step(s), cross entropy on all data is 0.0163385
After 2000 training step(s), cross entropy on all data is 0.00907547
After 3000 training step(s), cross entropy on all data is 0.00714436
After 4000 training step(s), cross entropy on all data is 0.00578471
w1= [[-1.96182752  2.58235407  1.68203771]
 [-3.46817183  1.06982315  2.11788988]]
w2= [[-1.82471502]
 [ 2.68546653]
 [ 1.41819501]]
